In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [17]:
# 🧩 Cell 1 — Setup & Imports (Kaggle)
import os, glob, cv2, random, math, numpy as np, shutil, json
from datetime import datetime
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

print("TensorFlow:", tf.__version__)


TensorFlow: 2.18.0


In [18]:
# 🧩 Cell 2 — Config (edit these as you like)
# If you know the exact path, set it directly (else auto-detects in next cell):
DATASET_ROOT = None  # e.g. "/kaggle/input/adityachandrasekhar-image-super-resolution"

TARGET_SIZE   = (128, 128)  # training patch size
DOWNSCALE     = 2           # synthetic LR scale if LR not provided
RANDOM_CROPS  = 0           # e.g., 4 or 8 for extra random crops per image
SEED          = 42

MODELS_TO_RUN = ["edsr", "vdsr"]   # train all three
LR            = 1e-4
EPOCHS        = 300                           # raise if you have more time
BATCH_SIZE    = 8

OUTPUT_DIR    = "/kaggle/working"

random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)


In [19]:
# 🧩 Cell 3 — Locate Dataset (auto-detect for https://www.kaggle.com/datasets/adityachandrasekhar/image-super-resolution)
def find_dataset_root():
    if DATASET_ROOT and os.path.isdir(DATASET_ROOT):
        return DATASET_ROOT
    candidates = sorted(glob.glob("/kaggle/input/*image*super*resolution*")) \
               + sorted(glob.glob("/kaggle/input/*super*resolution*")) \
               + sorted(glob.glob("/kaggle/input/*"))
    IMG_EXTS = (".jpg",".jpeg",".png",".bmp")
    for root in candidates:
        if os.path.isdir(root):
            for d,_,f in os.walk(root):
                if any(x.lower().endswith(IMG_EXTS) for x in f):
                    return root
    return None

DATA_ROOT = find_dataset_root()
assert DATA_ROOT is not None, "No dataset found under /kaggle/input. Add the dataset to your notebook."
print("DATA_ROOT:", DATA_ROOT)


DATA_ROOT: /kaggle/input/image-super-resolution


In [20]:
# 🧩 Cell 4 — Detect LR/HR folders if present (else we will synthesize LR)
def find_lr_hr_dirs(root):
    lr_names = ["LR","lr","Low","low","low_res","lowresolution"]
    hr_names = ["HR","hr","High","high","high_res","highresolution"]
    for base, dirs, _ in os.walk(root):
        s = set(dirs)
        for l in lr_names:
            for h in hr_names:
                if l in s and h in s:
                    lr_dir, hr_dir = os.path.join(base,l), os.path.join(base,h)
                    if len(glob.glob(os.path.join(lr_dir,"*"))) and len(glob.glob(os.path.join(hr_dir,"*"))):
                        return lr_dir, hr_dir
    return None, None

LR_DIR, HR_DIR = find_lr_hr_dirs(DATA_ROOT)
print("LR_DIR:", LR_DIR)
print("HR_DIR:", HR_DIR)

IMG_EXTS = (".jpg",".jpeg",".png",".bmp")
def list_images_recursive(root):
    out = []
    for d,_,f in os.walk(root):
        for fn in f:
            if fn.lower().endswith(IMG_EXTS):
                out.append(os.path.join(d,fn))
    return sorted(out)

if HR_DIR and LR_DIR:
    hr_paths = list_images_recursive(HR_DIR)
    lr_paths = list_images_recursive(LR_DIR)

    def stem(p):
        s = os.path.splitext(os.path.basename(p))[0].lower()
        for tag in ["_x2","_x3","_x4","-x2","-x3","-x4","@2x","@3x","@4x"]:
            s = s.replace(tag,"")
        return s

    lr_map = {stem(p): p for p in lr_paths}
    pairs = [(lr_map[stem(h)], h) for h in hr_paths if stem(h) in lr_map]
    if not pairs:
        print("Could not pair LR/HR by filenames; will synthesize LR from HR.")
        pairs = None
else:
    hr_paths = list_images_recursive(DATA_ROOT)
    pairs = None

print("Total HR images:", len(hr_paths))
if pairs is not None:
    print("Total LR/HR pairs:", len(pairs))


LR_DIR: /kaggle/input/image-super-resolution/dataset/Raw Data/low_res
HR_DIR: /kaggle/input/image-super-resolution/dataset/Raw Data/high_res
Total HR images: 855
Total LR/HR pairs: 855


In [21]:
# 🧩 Cell 5 — Preprocessing utilities
def read_rgb(path):
    im = cv2.imread(path, cv2.IMREAD_COLOR)
    if im is None: return None
    return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

def ensure_min_size(img, size):
    th, tw = size
    h, w = img.shape[:2]
    if h < th or w < tw:
        scale = max(th/h, tw/w)
        img = cv2.resize(img, (int(w*scale)+1, int(h*scale)+1), interpolation=cv2.INTER_CUBIC)
    return img

def center_crop(img, size):
    img = ensure_min_size(img, size)
    h, w = img.shape[:2]
    th, tw = size
    y = (h - th)//2
    x = (w - tw)//2
    return img[y:y+th, x:x+tw]

def random_crop(img, size):
    img = ensure_min_size(img, size)
    h, w = img.shape[:2]
    th, tw = size
    y = random.randint(0, h - th)
    x = random.randint(0, w - tw)
    return img[y:y+th, x:x+tw]

def synthesize_lr(hr_img, downscale=DOWNSCALE):
    h, w = hr_img.shape[:2]
    small = cv2.resize(hr_img, (max(1,w//downscale), max(1,h//downscale)), interpolation=cv2.INTER_AREA)
    lr    = cv2.resize(small, (w, h), interpolation=cv2.INTER_CUBIC)
    return lr


In [22]:
# 🧩 Cell 6 — Build dataset (X=LR, Y=HR) from real LR/HR or synthetic LR
def build_dataset_from_pairs(pairs):
    X, Y = [], []
    for lp, hp in pairs:
        lr = read_rgb(lp); hr = read_rgb(hp)
        if lr is None or hr is None: continue

        crop_fns = [center_crop] + ([random_crop]*RANDOM_CROPS if RANDOM_CROPS>0 else [])
        for fn in crop_fns:
            lr_c = cv2.resize(fn(lr, TARGET_SIZE), TARGET_SIZE, interpolation=cv2.INTER_CUBIC)
            hr_c = fn(hr, TARGET_SIZE)
            X.append(lr_c.astype(np.float32)/255.0)
            Y.append(hr_c.astype(np.float32)/255.0)
    return np.asarray(X, np.float32), np.asarray(Y, np.float32)

def build_dataset_from_hr(hr_paths):
    X, Y = [], []
    for hp in hr_paths:
        hr = read_rgb(hp)
        if hr is None: continue

        crop_fns = [center_crop] + ([random_crop]*RANDOM_CROPS if RANDOM_CROPS>0 else [])
        for fn in crop_fns:
            hr_c = fn(hr, TARGET_SIZE)
            lr_c = synthesize_lr(hr_c, DOWNSCALE)
            X.append(lr_c.astype(np.float32)/255.0)
            Y.append(hr_c.astype(np.float32)/255.0)
    return np.asarray(X, np.float32), np.asarray(Y, np.float32)

if pairs is not None:
    X, Y = build_dataset_from_pairs(pairs)
else:
    X, Y = build_dataset_from_hr(hr_paths)

print("LR shape:", X.shape, "HR shape:", Y.shape)

lr_train, lr_val, hr_train, hr_val = train_test_split(X, Y, test_size=0.2, random_state=SEED)
print("Train:", lr_train.shape, hr_train.shape, " Val:", lr_val.shape, hr_val.shape)


LR shape: (855, 128, 128, 3) HR shape: (855, 128, 128, 3)
Train: (684, 128, 128, 3) (684, 128, 128, 3)  Val: (171, 128, 128, 3) (171, 128, 128, 3)


In [23]:
# 🧩 Cell 7 — Models (SRCNN / EDSR-lite / VDSR-lite) and utilities
def build_srcnn(input_shape=(128,128,3)):
    inp = Input(shape=input_shape)
    x = Conv2D(64, 9, padding="same", activation="relu")(inp)
    x = Conv2D(32, 5, padding="same", activation="relu")(x)
    x = Conv2D(3,  5, padding="same")(x)
    return Model(inp, x, name="SRCNN")

def edsr_res_block(x, f):
    skip = x
    x = Conv2D(f, 3, padding="same", activation="relu")(x)
    x = Conv2D(f, 3, padding="same")(x)
    return Add()([skip, x])

def build_edsr(input_shape=(128,128,3), num_blocks=16, f=64):
    inp = Input(shape=input_shape)
    x = Conv2D(f, 3, padding="same")(inp)
    for _ in range(num_blocks):
        x = edsr_res_block(x, f)
    x = Conv2D(3, 3, padding="same")(x)
    return Model(inp, x, name="EDSR-lite")

def build_vdsr(input_shape=(128,128,3), depth=20, f=64):
    inp = Input(shape=input_shape)
    x = Conv2D(f, 3, padding="same", activation="relu")(inp)
    for _ in range(depth-2):
        x = Conv2D(f, 3, padding="same", activation="relu")(x)
    x = Conv2D(3, 3, padding="same")(x)
    return Model(inp, x, name="VDSR-lite")

def get_model(name, input_shape=(128,128,3)):
    n = name.lower()
    if n == "srcnn": return build_srcnn(input_shape)
    if n == "edsr":  return build_edsr(input_shape)
    if n == "vdsr":  return build_vdsr(input_shape)
    raise ValueError("Choose from {'srcnn','edsr','vdsr'}")

def psnr_metric(y_true, y_pred):
    return tf.image.psnr(y_true, y_pred, max_val=1.0)


In [24]:
# 🧩 Cell 8 — Train ALL 3 models (SRCNN, EDSR, VDSR) sequentially; save best+final for each
all_model_records = []   # keep paths for zipping later

for model_name in MODELS_TO_RUN:
    print(f"\n==============================")
    print(f"🔹 Training {model_name.upper()} ...")
    print(f"==============================")

    model = get_model(model_name, input_shape=(TARGET_SIZE[0], TARGET_SIZE[1], 3))
    model.compile(optimizer=Adam(LR), loss="mse", metrics=[psnr_metric])
    model.summary()

    best_path  = os.path.join(OUTPUT_DIR, f"{model_name}_best.h5")
    final_path = os.path.join(OUTPUT_DIR, f"{model_name}_final.h5")

    callbacks = [
        ModelCheckpoint(best_path, monitor="val_psnr_metric", mode="max", save_best_only=True, verbose=1),
        ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=8, min_lr=1e-6, verbose=1),
        EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1),
    ]

    history = model.fit(
        lr_train, hr_train,
        validation_data=(lr_val, hr_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
        verbose=1
    )

    model.save(final_path)
    print(f"✅ Saved {model_name.upper()} -> best: {best_path}, final: {final_path}")

    # save a couple previews per model
    preds_dir = os.path.join(OUTPUT_DIR, f"preds_{model_name}")
    os.makedirs(preds_dir, exist_ok=True)

    def save_preview(lr, hr, pred, idx):
        def to_u8(x): return np.clip((x*255.0).round(),0,255).astype(np.uint8)
        lr_u, hr_u, pd_u = to_u8(lr), to_u8(hr), to_u8(pred)
        cv2.imwrite(os.path.join(preds_dir,f"{idx:03d}_lr.png"),  cv2.cvtColor(lr_u, cv2.COLOR_RGB2BGR))
        cv2.imwrite(os.path.join(preds_dir,f"{idx:03d}_hr.png"),  cv2.cvtColor(hr_u, cv2.COLOR_RGB2BGR))
        cv2.imwrite(os.path.join(preds_dir,f"{idx:03d}_sr.png"),  cv2.cvtColor(pd_u, cv2.COLOR_RGB2BGR))

    for i in range(min(3, lr_val.shape[0])):  # 3 previews per model
        pred = model.predict(lr_val[i:i+1], verbose=0)[0]
        save_preview(lr_val[i], hr_val[i], pred, i)

    all_model_records.append({
        "name": model_name,
        "best": best_path,
        "final": final_path,
        "preds_dir": preds_dir
    })

print("\nAll models trained and saved.")



🔹 Training EDSR ...


Model: "EDSR-lite"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 128, 128,  │      1,792 │ input_layer_2[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 128, 128,  │     36,928 │ conv2d_6[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 128, 128,  │     36,928 │ conv2d_7[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 128, 128,  │          0 │ conv2d_6[0][0],   │
│                     │ 64)               │            │ conv2d_8[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 128, 128,  │     36,928 │ add[0][0]         │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 128, 128,  │     36,928 │ conv2d_9[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 128, 128,  │          0 │ add[0][0],        │
│                     │ 64)               │            │ conv2d_10[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 128, 128,  │     36,928 │ add_1[0][0]       │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 128, 128,  │     36,928 │ conv2d_11[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 128, 128,  │          0 │ add_1[0][0],      │
│                     │ 64)               │            │ conv2d_12[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 128, 128,  │     36,928 │ add_2[0][0]       │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 128, 128,  │     36,928 │ conv2d_13[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 128, 128,  │          0 │ add_2[0][0],      │
│                     │ 64)               │            │ conv2d_14[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 128, 128,  │     36,928 │ add_3[0][0]       │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_16 (Conv2D)  │ (None, 128, 128,  │     36,928 │ conv2d_15[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 128, 128,  │          0 │ add_3[0][0],    

 Total params: 1,185,219 (4.52 MB)

 Trainable params: 1,185,219 (4.52 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_40 (Conv2D)              │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_42 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_43 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_44 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_45 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_46 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_47 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_48 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_49 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_50 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_51 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_52 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_53 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_54 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_55 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_56 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_57 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_58 (Conv2D)              │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_59 (Conv2D)              │ (None, 128, 128, 3)    │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 668,227 (2.55 MB)

 Trainable params: 668,227 (2.55 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/300
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - loss: 0.1458 - psnr_metric: 10.8533
Epoch 1: val_psnr_metric improved from -inf to 18.38355, saving model to /kaggle/working/vdsr_best.h5
86/86 ━━━━━━━━━━━━━━━━━━━━ 25s 177ms/step - loss: 0.1450 - psnr_metric: 10.8941 - val_loss: 0.0195 - val_psnr_metric: 18.3836 - learning_rate: 1.0000e-04
Epoch 2/300
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - loss: 0.0159 - psnr_metric: 19.1698
Epoch 2: val_psnr_metric improved from 18.38355 to 21.40540, saving model to /kaggle/working/vdsr_best.h5
86/86 ━━━━━━━━━━━━━━━━━━━━ 13s 149ms/step - loss: 0.0159 - psnr_metric: 19.1761 - val_loss: 0.0100 - val_psnr_metric: 21.4054 - learning_rate: 1.0000e-04
Epoch 3/300
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - loss: 0.0076 - psnr_metric: 22.2484
Epoch 3: val_psnr_metric improved from 21.40540 to 24.23844, saving model to /kaggle/working/vdsr_best.h5
86/86 ━━━━━━━━━━━━━━━━━━━━ 13s 151ms/step - loss: 0.0076 - psnr_metric: 22.2549 - val_loss: 0.0049 - val_

In [ ]:
# 🧩 Cell 9 — Zip ALL outputs (models + previews) into one bundle for download
ts = datetime.now().strftime("%Y%m%d-%H%M%S")
bundle_dir = os.path.join(OUTPUT_DIR, f"sr_all_models_bundle_{ts}")
os.makedirs(bundle_dir, exist_ok=True)

# Copy files
for rec in all_model_records:
    # models
    for p in [rec["best"], rec["final"]]:
        if p and os.path.isfile(p):
            shutil.copy2(p, bundle_dir)
    # previews
    if os.path.isdir(rec["preds_dir"]):
        shutil.copytree(rec["preds_dir"], os.path.join(bundle_dir, os.path.basename(rec["preds_dir"])))

# metadata
meta = {
    "models": all_model_records,
    "target_size": list(TARGET_SIZE),
    "downscale": DOWNSCALE,
    "random_crops": RANDOM_CROPS,
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "learning_rate": float(LR),
    "created_at": ts,
}
with open(os.path.join(bundle_dir, "run_meta.json"), "w") as f:
    json.dump(meta, f, indent=2)

zip_path = os.path.join(OUTPUT_DIR, f"sr_all_models_{ts}")
shutil.make_archive(zip_path, "zip", bundle_dir)

print("Zipped to:", zip_path + ".zip")
print("Bundle contents:")
for root, dirs, files in os.walk(bundle_dir):
    for name in files:
        print("-", os.path.relpath(os.path.join(root, name), bundle_dir))
